In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [91]:
customers = pd.read_csv("/content/drive/MyDrive/Customers.csv")
transactions = pd.read_csv("/content/drive/MyDrive/Transactions.csv")
products = pd.read_csv("/content/drive/MyDrive/Products.csv")
ct = pd.merge(customers, transactions, on="CustomerID", how="inner")
df=pd.merge(ct, products, on="ProductID", how="inner")

In [92]:
df

,CustomerID,CustomerName,Region,SignupDate,TransactionID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,ProductName,Category,Price_y
0,C0001,Lawrence Carroll,South America,2022-07-10,T00015,P054,2024-01-19 3:12:55,2,114.60,57.30,SoundWave Cookbook,Books,57.30
1,C0001,Lawrence Carroll,South America,2022-07-10,T00932,P022,2024-09-17 9:01:18,3,412.62,137.54,HomeSense Wall Art,Home Decor,137.54
2,C0001,Lawrence Carroll,South America,2022-07-10,T00085,P096,2024-04-08 0:01:00,2,614.94,307.47,SoundWave Headphones,Electronics,307.47
3,C0001,Lawrence Carroll,South America,2022-07-10,T00445,P083,2024-05-07 3:11:44,2,911.44,455.72,ActiveWear Smartwatch,Electronics,455.72
4,C0001,Lawrence Carroll,South America,2022-07-10,T00436,P029,2024-11-02 17:04:16,3,1300.92,433.64,TechPro Headphones,Electronics,433.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,C0200,Kelly Cross,Asia,2023-06-11,T00518,P034,2024-12-11 3:05:50,4,868.20,217.05,HomeSense T-Shirt,Clothing,217.05
996,C0200,Kelly Cross,Asia,2023-06-11,T00091,P057,2024-04-27 19:06:20,1,239.70,239.70,ActiveWear Smartphone,Electronics,239.70
997,C0200,Kelly Cross,Asia,2023-06-11,T00731,P061,2024-07-15 20:36:28,4,627.84,156.96,HomeSense Desk Lamp,Home Decor,156.96
998,C0200,Kelly Cross,Asia,2023-06-11,T00771,P048,2024-09-10 9:50:48,4,1665.60,416.40,TechPro Cookbook,Books,416.40


In [93]:
df['Brand'] = df['ProductName'].str.split().str[0]
df['Brand'].value_counts()

,count
Brand,
ActiveWear,231
SoundWave,210
TechPro,156
HomeSense,152
BookWorld,132
ComfortLiving,119


In [94]:
df.drop(['CustomerName','SignupDate','Price_x','Price_y','ProductName','ProductID',"Region"],inplace=True,axis=1)

In [95]:
df

,CustomerID,TransactionID,TransactionDate,Quantity,TotalValue,Category,Brand
0,C0001,T00015,2024-01-19 3:12:55,2,114.60,Books,SoundWave
1,C0001,T00932,2024-09-17 9:01:18,3,412.62,Home Decor,HomeSense
2,C0001,T00085,2024-04-08 0:01:00,2,614.94,Electronics,SoundWave
3,C0001,T00445,2024-05-07 3:11:44,2,911.44,Electronics,ActiveWear
4,C0001,T00436,2024-11-02 17:04:16,3,1300.92,Electronics,TechPro
...,...,...,...,...,...,...,...
995,C0200,T00518,2024-12-11 3:05:50,4,868.20,Clothing,HomeSense
996,C0200,T00091,2024-04-27 19:06:20,1,239.70,Electronics,ActiveWear
997,C0200,T00731,2024-07-15 20:36:28,4,627.84,Home Decor,HomeSense
998,C0200,T00771,2024-09-10 9:50:48,4,1665.60,Books,TechPro


In [97]:
customer_spending = df.groupby(['CustomerID']).agg(total_spend=('TotalValue', 'sum'),avg_spend=('TotalValue', 'mean')).reset_index()

brand_counts = df.groupby(['CustomerID', 'Brand']).size().unstack(fill_value=0)

category_counts = df.groupby(['CustomerID', 'Category']).size().unstack(fill_value=0)

final_df = customer_spending.merge(brand_counts, on="CustomerID", how="left").merge(category_counts, on="CustomerID", how="left")
final_df


,CustomerID,total_spend,avg_spend,ActiveWear,BookWorld,ComfortLiving,HomeSense,SoundWave,TechPro,Books,Clothing,Electronics,Home Decor
0,C0001,3354.52,670.904000,1,0,0,1,2,1,1,0,3,1
1,C0002,1862.74,465.685000,0,2,1,0,0,1,0,2,0,2
2,C0003,2725.38,681.345000,4,0,0,0,0,0,0,1,1,2
3,C0004,5354.88,669.360000,2,2,0,0,1,3,3,0,2,3
4,C0005,2034.24,678.080000,1,0,1,0,0,1,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,C0196,4982.88,1245.720000,3,0,1,0,0,0,1,1,0,2
195,C0197,1928.65,642.883333,1,1,0,0,1,0,0,0,2,1
196,C0198,931.83,465.915000,0,0,1,1,0,0,0,1,1,0
197,C0199,1979.28,494.820000,1,1,1,1,0,0,0,0,2,2


In [98]:
ids=final_df['CustomerID']
final_df.drop(['CustomerID'],inplace=True,axis=1)

In [99]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(final_df)

In [100]:
cosine_sim = cosine_similarity(scaled_features)

customer_ids = ids
cosine_sim_df = pd.DataFrame(cosine_sim, index=customer_ids, columns=customer_ids)

In [101]:
lookalike_map = {}

for cust_id in ids[:20]:
  similar_customers = cosine_sim_df.loc[cust_id].nlargest(4).iloc[1:4]  # Top 3 excluding self
  similar_customers_rounded = list(zip(similar_customers.index, similar_customers.values.round(2)))
  lookalike_map[cust_id] = similar_customers_rounded

lookalike_df = pd.DataFrame(list(lookalike_map.items()), columns=["CustomerID", "Similar_Customers"])
lookalike_df


,CustomerID,Similar_Customers
0,C0001,"[(C0018, 0.7), (C0190, 0.63), (C0045, 0.63)]"
1,C0002,"[(C0030, 0.72), (C0029, 0.67), (C0133, 0.66)]"
2,C0003,"[(C0152, 0.74), (C0085, 0.66), (C0181, 0.65)]"
3,C0004,"[(C0100, 0.76), (C0118, 0.75), (C0065, 0.73)]"
4,C0005,"[(C0130, 0.78), (C0007, 0.72), (C0192, 0.71)]"
5,C0006,"[(C0171, 0.68), (C0158, 0.68), (C0085, 0.68)]"
6,C0007,"[(C0120, 0.76), (C0020, 0.76), (C0095, 0.74)]"
7,C0008,"[(C0098, 0.75), (C0179, 0.71), (C0075, 0.64)]"
8,C0009,"[(C0033, 0.88), (C0083, 0.82), (C0058, 0.82)]"
9,C0010,"[(C0056, 0.82), (C0077, 0.81), (C0061, 0.79)]"


In [102]:
final_df['CustomerID'] = ids

In [108]:
customers_of_interest = ['C0013','C0183', 'C0028', 'C0105']

# Get the indices of the rows where CustomerID is in the list
indices = final_df[final_df['CustomerID'].isin(customers_of_interest)].index

# Fetch the rows using iloc
filtered_transactions = final_df.iloc[indices]
filtered_transactions

,total_spend,avg_spend,ActiveWear,BookWorld,ComfortLiving,HomeSense,SoundWave,TechPro,Books,Clothing,Electronics,Home Decor,CustomerID
12,6000.56,857.222857,3,1,0,2,1,0,1,3,3,0,C0013
27,6819.57,852.446250,4,1,0,2,1,0,2,2,3,1,C0028
104,4807.45,801.241667,3,0,0,1,1,1,1,2,3,0,C0105
181,3850.22,770.044000,2,0,0,2,1,0,1,2,2,0,C0183


In [109]:
customers_of_interest = ['C0018','C0148', 'C0114', 'C0001']

# Get the indices of the rows where CustomerID is in the list
indices = final_df[final_df['CustomerID'].isin(customers_of_interest)].index

# Fetch the rows using iloc
filtered_transactions = final_df.iloc[indices]
filtered_transactions

,total_spend,avg_spend,ActiveWear,BookWorld,ComfortLiving,HomeSense,SoundWave,TechPro,Books,Clothing,Electronics,Home Decor,CustomerID
0,3354.52,670.904,1,0,0,1,2,1,1,0,3,1,C0001
17,4781.85,956.370,1,0,0,1,2,1,2,0,2,1,C0018
113,5780.43,1156.086,1,0,0,1,2,1,1,1,2,1,C0114
147,5457.79,1091.558,1,0,0,1,2,1,1,0,2,2,C0148


In [111]:
lookalike_df.to_csv('Jay_Joshi_Lookalike.csv', index=False)